# **Demo - Oscilloscopio**

Il notebook mostra come sia possible riprodurre un oscilloscopio interattivo per visualizzare in continua delle acquisizioni dell'\texttt{ADC}. Lo script apre una finestra interattiva in cui è possibile interagire con l'oscilloscopio nel seguente modo:

* Mouse scroll: permtte di controllare lo zoom biassiale quando il mouse si trova nel grafico, uniassiale quando si trova su uno degli assi.
* Tasto `'a'` forza un *autoscale* del grafico
* Tasto `'.'` visualizza le tracce dell'oscilloscopio come punti
* Tasto `'-'` visualizza le tracce dell'oscilloscopio come linee
* Tasto `'x'` esporta l'ultima misura su file (nome del file da indicare su prompt della shell python)
* Tasto `'space'` entra/esce da pausa nell'acquisizione
* Tasto `'enter'` fa una singola acquisizione
* Tasto `'esc'` termina il programma

I parametri di acquisizione e le forme d'onda del generatore vanno impostate nel codice con le semplici istruzioni indicate sotto.

In [1]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione
ad2 = tdwf.AD2()

ad2.vdd = +5
ad2.vss = -5
ad2.power(True)
#   2. Configurazione generatore di funzioni
wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.ampl = 5
wavegen.w1.freq = 20
#wavegen.w1.offs = 0.0
wavegen.w1.phi = 0.0
wavegen.w1.func = tdwf.funcSquare
wavegen.w2.config(ampl=1, freq=1000000.1, phi=0.0, func=tdwf.funcSine, duty=50)
wavegen.w2.sync()
wavegen.w1.start()
#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs = 1e2
scope.npt = 8000
scope.ch1.rng = 5
scope.ch2.rng = 5
scope.ch1.avg = True
scope.ch2.avg = True
scope.trig(True, level=0.0, sour=tdwf.trigsrcCh1, delay=0, hist=0.1)

# -[Funzioni di gestione eventi]-----------------------------------------------

def on_close(event):
    global flag_run
    flag_run = False

def on_scroll(event):
    kk = 1.5  # fattore di zoom/dezoom
    # [1] Calcolo delle coordinate del mouse rispetto agli assi
    x0, x1 = ax.get_xlim()  # limiti asse x
    y0, y1 = ax.get_ylim()  # limiti asse y
    figw, figh = fig.get_size_inches() * fig.dpi  # calcola dimensioni finestra
    box = ax.get_position()  # posizione assi nella finestra
    xdata = (event.x/figw-box.x0)/(box.x1-box.x0)*(x1-x0)+x0
    ydata = (event.y/figh-box.y0)/(box.y1-box.y0)*(y1-y0)+y0
    # [2] del fattore di zoom
    if event.button == 'up':
        factor = 1 / kk
    elif event.button == 'down':
        factor = kk
    else:
        return
    # [3] calcolo delle nuove coordinate limite degli assi
    newdx = (x1-x0) * factor  # nuovo span asse x
    relx = (x1 - xdata) / (x1-x0)  # posizione relativa mouse nello span
    newdy = (y1-y0) * factor  # nuovo span assey
    rely = (y1 - ydata) / (y1-y0)  # posizinoe relativa mouse nello span
    if xdata > x0:  # zoom x
        ax.set_xlim([xdata - newdx * (1-relx), xdata + newdx * (relx)])
    if ydata > y0:   # zoom y
        ax.set_ylim([ydata - newdy * (1-rely), ydata + newdy * (rely)])
    # [4] aggiorna la figura
    fig.canvas.draw()
    fig.canvas.flush_events()

def on_key(event):
    global flag_run, flag_acq, hp1, hp2
    if event.key == 'a':  # autoscale
        ax.set_xlim([tt.min(), tt.max()])
        ax.set_ylim([min(min1.min(), min2.min()), max(max1.max(), max2.max())])
    if event.key == 'x':  # export su file
        filename = input("Esporta dati su file: ")
        data = np.column_stack((scope.time.vals, scope.ch1.vals, scope.ch2.vals))
        info = f"Acquisizione Analog Discovery 2\nTimestamp {scope.time.t0}\ntime\tch1\tch2"
        np.savetxt(filename, data, delimiter='\t', header=info)
    if event.key == '.':  # plot a punti
        hp1.set_linestyle("")
        hp1.set_marker(".")
        hp2.set_linestyle("")
        hp2.set_marker(".")
    if event.key == '-':  # plot a linea
        hp1.set_linestyle("-")
        hp1.set_marker("")
        hp2.set_linestyle("-")
        hp2.set_marker("")
    if event.key == ' ':  # run/pausa
        flag_acq = not flag_acq
    if event.key == 'enter':  # acqusizione singola
        flag_acq = False
        scope.sample()
    if event.key == 'escape':  # termina programma
        flag_run = False


# -[Ciclo di misura]-----------------------------------------------------------
#   1. Creazione figura e link agli eventi
fig, ax = plt.subplots(figsize=(12, 6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect('scroll_event', on_scroll)
fig.canvas.mpl_connect('key_press_event', on_key)
#   2. Creazione dei vettori dei tempi
# (troppi punti, media uno ogni 4)
tt = 1e3*scope.time.vals.reshape(-1, 4).mean(axis=1)
# (vettore degli errori: buffer più corto di un fattore 4)
tte = np.repeat(tt[::4], 2)
tte = np.append(tte[1:]-8e3*scope.time.dt, tte[-1]+8e3*scope.time.dt)
#   3. Ciclo di misura
flag_first = True
flag_acq = True
flag_run = True
while flag_run:
    if flag_acq:  # SE la misura è attiva (space)
        scope.sample()
    # Calcolo dei vettori da visualizzare (vengono fatte alcune medie)
    min1 = np.repeat(scope.ch1.min.reshape(-1, 2).mean(axis=1), 2)
    max1 = np.repeat(scope.ch1.max.reshape(-1, 2).mean(axis=1), 2)
    min2 = np.repeat(scope.ch2.min.reshape(-1, 2).mean(axis=1), 2)
    max2 = np.repeat(scope.ch2.max.reshape(-1, 2).mean(axis=1), 2)
    ch1 = scope.ch1.vals.reshape(-1, 4).mean(axis=1)
    ch2 = scope.ch2.vals.reshape(-1, 4).mean(axis=1)
    if flag_first:
        # Prima esecuzione: creazione grafici
        flag_first = False
        hp1, = plt.plot(tt, ch1, "-", label="Ch1", color="tab:orange")
        hp2, = plt.plot(tt, ch2, "-", label="Ch2", color="tab:blue")
        hp3 = plt.fill_between(tte, min1, max1, color='tab:orange', alpha=0.3)
        hp4 = plt.fill_between(tte, min2, max2, color='tab:blue', alpha=0.3)
        path1 = hp3.get_paths()[0]
        path2 = hp4.get_paths()[0]
        plt.legend()
        plt.grid(True)
        plt.xlabel("Time [msec]", fontsize=15)
        plt.ylabel("Signal [V]", fontsize=15)
        plt.title("User interaction: a|-|.|x|space|enter|escape|scroll")
        plt.show(block=False)
        plt.tight_layout()
        ax.set_xlim([tt.min(), tt.max()])
        #ax.set_ylim([min(min1.min(), min2.min()), max(max1.max(), max2.max())])
    else:
        # Esecuzioni successive: aggiornamento grafici
        hp1.set_ydata(ch1)
        hp2.set_ydata(ch2)
        tmp1 = np.concatenate(([max1[0]], min1, [max1[-1]], max1[::-1], [max1[0]]))
        path1.vertices[:, 1] = tmp1
        tmp2 = np.concatenate(([max2[0]], min2, [max2[-1]], max2[::-1], [max2[0]]))
        path2.vertices[:, 1] = tmp2
        fig.canvas.draw()
        fig.canvas.flush_events()

#   4. Chiude figura e libera AD2
plt.close(fig)
ad2.close()

Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321AD14E6, hdwf=1] connesso!
Configurazione #1


: 

## Qualche spiegazione per chi vuole approfondire ...

In questa sezione illustriamo alcuni aspetti del codice che non è necessario conoscere, ma che magari potrebbe incuriosire alcuni di voi.

### 1. Gestione degli eventi

Lo script è "interattivo" ed è in grado di reagire a varie "eventi", come per esempio la pressione di un tasto da parte dell'utente. Le funzioni di gestione degli eventi sono riportate in fondo allo script e sono le seguenti

* `on_close` $\implies$ interrompe loop di esecuzione quando viene chiusa la finestra.
* `on_scroll` $\implies$ gestisce gli eventi della rotella del mouse.
* `on_key` $\implies$ gestisce gli eventi della tastiera.

Il modo in cui queste funzioni sono "collegate" agli eventi è tramite le seguenti funzioni del modulo `matplotlib`

        fig.canvas.mpl_connect("close_event", on_close)
        fig.canvas.mpl_connect('scroll_event', on_scroll)
        fig.canvas.mpl_connect('key_press_event', on_key)

che per esempio fanno sì che la funzione `on_key` venga chiamata quando viene premuto in tasto nella finestra `fig`. Queste permettono di gestire varie funzionalità. Per esempio il ciclo di misura è apparentemente perpetuo in quanto ha questa struttura

        flag_run = True
        while flag_run:
                ...

e il booleano di controllo `flag_run` non viene modificato dentro il ciclo. Tuttavia, in realtà viene modificato quando viene premuto il tasto `escape`. Questo evento infatti triggera una chiamata a `on_key`, e l'informazione del tasto premuto può essere trovata nell'argomento `event.key`.

### 2. Scroll del mouse

La funzione di gestione dello "scroll" (rotella del mouse) merita un discorso a latere. In questo caso le coordinate del mouse sono disponibili come `event.xdata` e  `event.ydata`. Tuttavia, queste coordinate sono solo utilizzabili *dentro gli assi*, mentre qui si voleva essere in grado di fare degli zoom uniassiali posizionando il mouse su uno dei due assi. Questo richiede di ricostruire le coordinate del mouse anche quando questo non si trova dentro i limiti degli assi. 

Per questo scopo, la funzione parte in realtà da `event.x` ed `event.y` che sono le coordinate del mouse (in pixel) rispetto alla finestra. Queste, assieme alle informazioni (facilmente recuperabili) della posizione degli assi nella finestra, permettono di ricostruire le coordinate del mouse rispetto agli assi *anche per posizioni che stanno fuori dagli assi*. Il resto della funzione effettua uno zoom per un certo fattore, tenendo fisso il punto in cui si trova il mouse.

### 3. Configurazione del generatore di funzioni

Per riferimento, ricordiamo quali sono le proprietà che è possibile regolare nella definizione del segnale fornito dalle uscite analogiche $\texttt{W1}$ e $\texttt{W2}$. 

| Proprietà | Descrizione | Valore di defaut |
| --------------- | --------------- | --------------- |
| `.ampl`   | ampiezza di picco | `1V`  |
| `.freq`   | frequenza | `1kHz`  |
| `.offs`   | *offset* della forma d'onda | `0V`  |
| `.duty`   | *duty cycle* | `50%`  |
| `.phi`   | fase dell'onda | `0deg`  |
| `.func`   | tipo di forma d'onda | `tdwf.funcSine`  |
| `.data`   | vettore di descrizione onde arbitrarie | `[]`  |

Si ricordano anche le principali forme d'onda disponibili

| Costante | Descrizione |
| --------------- | --------------- |
| `tdwf.funcDC` | Onda nulla $\implies$ l'output costante e uguale all'{\em offset} |
| `tdwf.funcSine` | Sinusoide (questa è la configurazione di default) |
| `tdwf.funcSquare` | Onda quadra, con {\em duty} controllato dal valore di `duty` |
| `tdwf.funcTriangle` | Onda triangolare, che con `duty` pul diventare un dente di sega | 
| `tdwf.funcCustom` | Onda arbitraria in base al contento dell'array `w#.data` |

### 4. Aggiornamento dei grafici

Lo script è scritto per permettere un aggionramento interattivo e rapido dei grafici. Questo richiede alcuni passaggi:

**Backend**. In primis, è necessario che il grafico venga plottato su una figura separata e non in-line. Questo è possibile impostando un "back-end" grafico diverso da quello standard per i notebook di Jupyter. Questo è garantito dall'istruzione 

        import matplotlib
        matplotlib.use('TkAgg')

dove `TkAgg` sta per `Tk`, una delle interfacce grafiche più note in python, più `Agg` (Anti-grain geometry). 

**Aggiornamento grafici**. Oltre a questo, il loop di misura dello script distingue fra la prima esecuzione (in cui vengono creati i grafici) e quelle successive (dove invece i grafici vengono *aggiornati*). La distinzione fra i due casi è possibile grazie al booleano `flag_first`. Alla prima esecuzione lo script "prende nota" dell'oggetto prodotto da `plot` e la salva in una variabile `hp1`

        hp1, = plt.plot(tt, ch1, "-", label="Ch1", color="tab:orange")

mentre nelle esecuzioni successive lo script va solo a modificare i dati plottati con la seguente istruzione

        hp1.set_ydata(ch1)

Questo approccio è computazionalmente più leggero di quello necessario a cancellare la figura e riplottare tutto da zero.

**Reshape e medie varie**. Alcuni passaggi nello script possono apparire oscuri, in particolare la manipolazione effettuata sui vettori di misura subito dopo l'istruzione `.sample()`. Se ricostruite l'algoritmo vi renderete conto che sono delle semplici medie. Il motivo per cui vengono fatte è molto semplice. Dato che lo strumento misura fino a 8192 punti e il vostro schermo non ha certo così tanti pixel, sarebbe uno spreco cercare di disegnarli tutti. Questo quindi deriva dal fatto che qui vogliamo *visualizzare* il dato. Se lo volessimo salvare su file probabilemente non vorremo fare queste medie (al più le farà chi analizza il dato, se crede). Se guardate bene, l'esportazione dei dati eseguita al premere di `x` salva il dato grezzo, e non quello mediato.

### 5. Ulteriori informazioni

**Min/Max e media** Quando non viene sfruttata tutta la banda di acquisizione di AD2, lo strumento fornisce ulteriori funzionalità che non sono sempre palese. Il problema è ovvio: AD2 acquisisce SEMPRE a 100MSa/s, ma che cosa fa con tutte le altre misure? 

* Una prima opzione è che possiamo chiedere ad AD2 di semplicemente prendere una misura ogni $N$, oppure di *mediare* le $N$ misure sull'hardware. Questa opzione viene attivata con 

        .avg = True / False

* AD2 fornisce automaticamente (poi magari si può ignorare ma il dato c'è ed è automatico) una informazione sui valori massimi e minimi osservati durante la misura. Nel caso in cui la misura procede ad alta velocità e/o non viene mediata, i massimi e minimi vengono sostanzialmente sempre misurati e li trovate fra i valori di misura. Nel caso in cui la misura procede abbastanza più lentamente ed è mediata con l'opzione citata prima,

        .ch#.min
        .ch#.max

saranno dei vettori che contengono i valori massimi e minimi incontrati da AD2 durante *tutte* le misure, che sono sempre effettuate a 100MSa/s. Si noti che questi sono dei *vettori* di valori, ma non sono densi quanto la misura richiesta e avremo un valore di minimo e uno di massimo ogni *quattro* prese dati. Per essere concreti `.ch1.vals` è lungo quattro volte `.ch1.min`.